In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [2]:
csv_file = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/\
002-obdataset/ann-r.csv'
test_file = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/\
002-obdataset/test.csv'
CLASSES_FILE = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/\
002-obdataset/classes.csv'
df = pd.read_csv(csv_file, names=['filename', 'x_min', 'y_min', 'x_max', 'y_max', 'class_name'])
df.head(10)

,filename,x_min,y_min,x_max,y_max,class_name
0,/home/kimsoohyun/00-Research/02-Graph/02-image...,0,96,1440,2612,clickable
1,/home/kimsoohyun/00-Research/02-Graph/02-image...,192,0,1248,726,clickable
2,/home/kimsoohyun/00-Research/02-Graph/02-image...,0,1318,1440,1656,clickable
3,/home/kimsoohyun/00-Research/02-Graph/02-image...,256,1424,1200,1656,clickable
4,/home/kimsoohyun/00-Research/02-Graph/02-image...,0,1736,1440,2612,clickable
5,/home/kimsoohyun/00-Research/02-Graph/02-image...,256,1842,1200,2152,clickable
6,/home/kimsoohyun/00-Research/02-Graph/02-image...,256,2232,1200,2612,clickable
7,/home/kimsoohyun/00-Research/02-Graph/02-image...,32,2660,341,2852,clickable
8,/home/kimsoohyun/00-Research/02-Graph/02-image...,992,2660,1344,2852,clickable
9,/home/kimsoohyun/00-Research/02-Graph/02-image...,0,96,1440,2612,clickable


In [3]:
path = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/03-models/retinanet/resnet50_csv_10.h5'

#print(model_path.save_)
model = models.load_model(path, backbone_name='resnet50')
model = models.convert_model(model)

Using TensorFlow backend.


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

In [4]:
THRES_SCORE = 0.35

def predict(image):
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    boxes /= scale
    return boxes, scores, labels
def draw_detections(image, boxes, scores, labels):
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < THRES_SCORE:
            break
        color = label_color(label)
        b = box.astype(int)
        draw_box(image, b, color=color)
        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(image, b, caption)
def show_detected_objects(image):
    #img_path = image_row.filename
    #image = read_image_bgr(img_path)
    boxes, scores, labels = predict(image)
    print(len(boxes[0]))
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    #true_box = [image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max]
    #draw_box(draw, true_box, color=(255, 0, 0))
    draw_detections(draw, boxes, scores, labels)
    #plt.axis('off')
    
    #plt.imshow(draw)
    #plt.show()
    return draw

In [5]:
labels_to_names = pd.read_csv(
  CLASSES_FILE,
  header=None
).T.loc[0].to_dict()

In [6]:
class VideoStreaming(object):                                                   
    def __init__(self):                                                         
        print("TEST1")                                                          
        self.cap = cv2.VideoCapture(0)                                          
        self.cap_res = self.resolution(self.cap, 1920, 1080)                    
        self.is_record = False                                                  
        self.recordingThread = None                                             
        self.out = None                                                         
    def __del__(self):                                                          
        self.cap.release()                                                      
                                                                                
    def resolution(self, capture, weight, height):                              
        capture.set(3, weight)                                                  
        capture.set(4, height)                                                  
        return capture                                                          
                                                                                
    def get_frame(self):                                                        
        ret, frame = self.cap_res.read()                                        
        return ret, frame                                                       
                                                                                
    def start_record(self, file_name):                                          
        self.is_record = True                                                   
        self.recordingThread = RecordingThread(file_name, self.cap)             
        self.recordingThread.start()                                            
                                                                                
    def stop_record(self):                                                      
        self.is_record = False                                                  
        if self.recordingThread != None:                                        
            self.recordingThread.stop()  

In [7]:
def get_fps():                                                                  
    cam = VideoStreaming()                                                      
                                                                                
    num_frames = 300                                                            
    print("Capturing {0} frames".format(num_frames))                            
                                                                                
    start = time.time()                                                         
    for i in range(0, num_frames):                                              
        ret, frame = cam.get_frame()#video.read()                               
    end = time.time()                                                           
                                                                                
    seconds = end - start                                                       
    print("Time taken: {0} seconds".format(seconds))                            
                                                                                
    fps = num_frames / seconds                                                  
    return fps 

In [8]:
cam = VideoStreaming()                                                      
print("START")                                                              
try:                                                                        
    while(True):                                                            
        ret, frame = cam.get_frame() 
        height = frame.shape[0]                                             
        width = frame.shape[1]                                      
        #vis = frame.copy()     
        vis = show_detected_objects(frame)

        cv2.imshow('frame', vis)                                                                            
        if cv2.waitKey(1) & 0xFF == ord('q'):                               
            break                                                           
except KeyboardInterrupt as e:                                              
    print("[CTRL-C] is captured!!")                                        
    cam.cap_res.release()                                                  
    cv2.destroyAllWindows()                                                




TEST1
START
300
300
300
300
[CTRL-C] is captured!!
